# 获取底盘反馈信息

产品开机后，下位机默认会持续向上位机反馈各类信息，你可以通过这些反馈信息来获取目前产品的工作状态。

通常来说，你需要连续获取下位机反馈的信息，但在本例程中，我们只获取一个由下位机反馈的 JSON 信息（注释掉或删除 break 这行即可连续获取反馈信息）。

选中以下的代码块，使用 Ctrl + Enter 运行该代码块，当它获取第一条完整的且T值为 1001 的 JSON 信息后会跳出循环并输出反馈信息，反馈信息包含当前车轮转速、IMU、云台角度（如果有安装的话）、机械臂角度（如果有安装的话）、电源电压等信息。

In [ ]:
from base_ctrl import BaseController
import json

# 用于检测树莓派的函数
def is_raspberry_pi5():
    with open('/proc/cpuinfo', 'r') as file:
        for line in file:
            if 'Model' in line:
                if 'Raspberry Pi 5' in line:
                    return True
                else:
                    return False

# 根据树莓派的型号来确定 GPIO 串口设备名称
if is_raspberry_pi5():
    base = BaseController('/dev/ttyAMA0', 115200)
else:
    base = BaseController('/dev/serial0', 115200)

# 使用无限循环来不断监听串口数据
while True:
    try:
        # 从串口读取一行数据，解码成 'utf-8' 格式的字符串，并尝试将其转换为 JSON 对象
        data_recv_buffer = json.loads(base.rl.readline().decode('utf-8'))
        # 检查解析出的数据中是否包含 'T' 键
        if 'T' in data_recv_buffer:
            # 如果 'T' 的值为 1001，则打印接收到的数据，并跳出循环
            if data_recv_buffer['T'] == 1001:
                print(data_recv_buffer)
                break
    # 如果在读取或处理数据时发生异常，则忽略该异常并继续监听下一行数据
    except:
        pass

## 非堵塞方式获取串口发来的 JSON 信息

以下的代码仅用于理解底层的串口读取 JSON 信息的原理，不可以执行下面的代码块。

In [ ]:
class ReadLine:
    # 构造函数，初始化 ReadLine 类的实例
    # s: 传入的串口对象，用于与串口进行通信。
    def __init__(self, s):
        self.buf = bytearray()  # 初始化一个字节数组，用于存储从串口读取但尚未处理的数据
        self.s = s  # 保存传入的串口对象，后续用于读取串口数据

	def readline(self):
		i = self.buf.find(b"\n") # 查找缓冲区中是否有换行符
		if i >= 0:
			r = self.buf[:i+1] # 如果有换行符，将换行符前的数据提取出来
			self.buf = self.buf[i+1:] # 更新缓冲区，去除已处理的数据
			return r
		while True:
			i = max(1, min(512, self.s.in_waiting)) # 获取可读取的字节数，最多512个字节
			data = self.s.read(i) # 从串口读取数据
			i = data.find(b"\n") # 查找换行符
			if i >= 0:
				r = self.buf + data[:i+1] # 如果找到换行符，将已读取的数据与缓冲区中的数据合并
				self.buf[0:] = data[i+1:] # 更新缓冲区，去除已处理的数据
				return r
			else:
				self.buf.extend(data) # 如果未找到换行符，将数据添加到缓冲区

- 该方法用于从串口读取数据，并返回一行完整的JSON数据（以换行符\n为分隔）。
- 如果缓冲区中已经存在完整的一行数据，则直接返回该行数据。
- 如果缓冲区中没有完整的一行数据，则通过 in_waiting 方法获取串口缓冲区中可读取的字节数，最多读取512字节。
- 从串口读取数据，并将其与缓冲区中的数据合并。
- 查找新读取数据中是否有换行符，如果有则提取完整的一行数据，并更新缓冲区。
- 如果没有换行符，则将新读取的数据添加到缓冲区，继续读取直到找到换行符为止。行符为止。

### 函数特性

- 非阻塞: 这个函数使用了非阻塞的读取方式，即使串口没有数据可读，也不会阻塞程序的执行，而是等待有数据可读时才读取。
- 高效: 采用了较小的缓冲区，每次最多读取512字节，减少了内存消耗，并且及时处理数据，避免了缓冲区溢出。
- 灵活性: 可以灵活地读取任意长度的数据，并且自动处理换行符分割的数据，非常适用于读取JSON等结构化数据。
- 可靠性: 在处理读取过程中考虑了各种情况，如缓冲区数据不足、读取数据中不含换行符等情况，保证了读取的准确性和稳定性。
- 这个函数非常适合用于实时读取串口中的JSON数据，尤其是在需要非阻塞读取的情况下。情况下。